In [ ]:
from qadataset import QADataset
from config import dataset_config, model_config

train_dataset = QADataset(dataset_config, split="train[:6]")
val_dataset = QADataset(dataset_config, split="train[:6]")

/home/atomwalk12/Dropbox (Old)/notes/vision/project/BeyondVisionQA/model.py:128: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  self.metric = load_metric("bertscore")
/home/atomwalk12/anaconda3/envs/questllama/lib/python3.9/site-packages/datasets/load.py:756: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Map:   0%|          | 0/38575 [00:00<?, ? examples/s]

Map:   0%|          | 0/38575 [00:00<?, ? examples/s]

: 

In [1]:
from model import get_model

model = get_model(quantization='8bit')

ImportError: cannot import name 'WUPMeasure' from partially initialized module 'model' (most likely due to a circular import) (/home/atomwalk12/Dropbox (Old)/notes/vision/project/BeyondVisionQA/model.py)

In [3]:
from trainer_blip2 import BLIP2ModelPLModule, BLIP2PLModule


hyperparameters = model_config['hyperparameters']
if model_config['target'] == 'blip2':
    module = BLIP2PLModule(hyperparameters, model, train_dataset, val_dataset)

In [4]:
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
import lightning as L

def train(module: L.LightningModule):
    hyperparams = model_config['hyperparameters']

    early_stop_callback = EarlyStopping(monitor="wup_measure", patience=3, verbose=False, mode="min")
    print(hyperparameters.values())
    trainer = L.Trainer(
            accelerator="gpu",
            devices=[0],
            max_epochs=200,
            accumulate_grad_batches=hyperparams.get("accumulate_grad_batches"),
            check_val_every_n_epoch=10,
            gradient_clip_val=hyperparams.get("gradient_clip_val"),
            precision="16-mixed",
            num_sanity_val_steps=0,
            default_root_dir=model_config['local_checkpoint_dir'],
            callbacks=[early_stop_callback],
    )

    trainer.fit(module)

train(module)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/atomwalk12/anaconda3/envs/questllama/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/genera

dict_values([20, 1.67, 1, 1.0, 8, 0.0001, 9, 1337, 1, './result', True, [0.9, 0.999], 0.05, 256])
Applying both Adam and Cosine Scheduler Warmup
Running warmup for a total of 0 steps


/home/atomwalk12/anaconda3/envs/questllama/lib/python3.9/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

/home/atomwalk12/anaconda3/envs/questllama/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 23.61 GiB of which 68.81 MiB is free. Process 60562 has 15.10 GiB memory in use. Including non-PyTorch memory, this process has 7.70 GiB memory in use. Of the allocated memory 6.96 GiB is allocated by PyTorch, and 285.13 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 